In [4]:
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy.types import Integer, String, Float, DateTime
from sqlalchemy.exc import SQLAlchemyError

# Step 1: 读取Excel文件
file_path = r"D:\workspace\Current project\shadow_bot\千百度辅助表\月聚水潭.xlsx"
df = pd.read_excel(file_path)

# 查看数据
print(df.head())

# Step 2: 创建数据库连接
db_username = 'root'  # 替换为你的数据库用户名
db_password = '111111'  # 替换为你的数据库密码
db_host = '192.168.10.207'  # 替换为你的数据库主机地址
db_port = '3306'  # 通常是3306
db_name = 'art_data_demo'  # 替换为你的数据库名称

engine = create_engine(f'mysql+mysqlconnector://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')
metadata = MetaData()

# Step 3: 自动创建表并插入数据到MySQL数据库
table_name = 'data_app_product_month'  # 替换为你的目标表名

# 定义数据类型映射
dtype_mapping = {
    'int64': Integer(),
    'float64': Float(),
    'object': String(255),
    'datetime64[ns]': DateTime()
}

# 生成SQLAlchemy兼容的数据类型字典
dtype = {col: dtype_mapping[str(dtype)] for col, dtype in df.dtypes.items()}

chunk_size = 8000  # 定义分块大小

try:
    with engine.connect() as connection:
        for start in range(0, len(df), chunk_size):
            end = start + chunk_size
            df_chunk = df.iloc[start:end]
            with connection.begin() as transaction:
                try:
                    df_chunk.to_sql(table_name, con=connection, if_exists='append', index=False, dtype=dtype)
                    transaction.commit()
                    print(f"Chunk {start} to {end} inserted successfully.")
                except SQLAlchemyError as e:
                    transaction.rollback()
                    print(f"Error inserting chunk {start} to {end}: {e}")
except SQLAlchemyError as e:
    print(f"Error with database connection: {e}")


d:\workspace\Current project\Art data view\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


     内部订单号                                     线上订单号  平台站点  \
0  9898633  2405215330796369208064850709600213003585   唯品会   
1  9898634                       2161922592666305673  淘宝天猫   
2  9898635  2405218671701369207505278768813653004133   唯品会   
3  9898636                    240522-399339702052739   拼多多   
4  9898637                       3899210547997352047  淘宝天猫   

                  下单时间          店铺名称   应付金额   已付金额   状态            款号  \
0  2024-05-22 00:00:10        伊伴-唯品会    0.0    0.0   取消   EG541191D73   
1  2024-05-22 00:00:27      伊伴-天猫旗舰店  225.1  225.1   取消   ER842384S73   
2  2024-05-22 00:00:52      伊伴桐乡-唯品会    0.0    0.0   取消   GA741092D28   
3  2024-05-22 00:00:44    千百度-拼多多旗舰店  263.0  263.0  已发货  C2132876WA45   
4  2024-05-22 00:01:04  千百度-天猫休闲鞋旗舰店  428.9  428.9  已发货   QN341826W57   

                商品编码  数量  登记数量  实退数量    成本价  
0   EG541191D7373240   1     0     0  101.0  
1   ER842384S7373245   1     0     0   78.0  
2   GA741092D2828245   1     0     0  102.0  
3  C

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
from PIL import Image as PILImage
import os

# 创建数据库连接（以MySQL为例）
# Step 2: 创建数据库连接
db_username = 'root'  # 替换为你的数据库用户名
db_password = '111111'  # 替换为你的数据库密码
db_host = '192.168.10.207'  # 替换为你的数据库主机地址
db_port = '3306'  # 通常是3306
db_name = 'art_data_demo'  # 替换为你的数据库名称

engine = create_engine(f'mysql+mysqlconnector://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# 编写SQL查询语句
query = 'SELECT * FROM data_app_product'

# 执行查询并将结果存储在DataFrame中
df = pd.read_sql(query, con=engine)




In [3]:

# 创建一个新的Excel工作簿和工作表
wb = Workbook()
ws = wb.active

# 将DataFrame写入Excel工作表
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# 插入图片到对应单元格
for row in range(2, len(df) + 2):  # 从第2行开始，因为第1行为标题行
    image_path = ws.cell(row=row, column=1).value  # 假设图片路径在第3列
    try:
        image_path_absolute = fr"D:\workspace\Current project\Art data view\web_test\static{image_path}"
    except:
        pass
    print(image_path_absolute)
    
    if image_path_absolute and os.path.exists(image_path_absolute):
        img = Image(image_path_absolute)
        img.width, img.height = 100, 100  # 调整图片大小
        img_cell = ws.cell(row=row, column=1).coordinate  # 假设图片插入到第4列
        ws.add_image(img, img_cell)

# 保存Excel文件
wb.save('output_with_images.xlsx')

D:\workspace\Current project\Art data view\web_test\static/img/QF542921W25.jpg
D:\workspace\Current project\Art data view\web_test\static/img/C0112071L01.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QA1522133W45.jpg
D:\workspace\Current project\Art data view\web_test\static/img/C1513157D01.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QA1042842SC1.jpg
D:\workspace\Current project\Art data view\web_test\static/img/C2922893L24.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QM542983WF2.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QA742087D73.jpg
D:\workspace\Current project\Art data view\web_test\static/img/C7732150L01.jpg
D:\workspace\Current project\Art data view\web_test\static/img/C2121646D45.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QF542921W26.jpg
D:\workspace\Current project\Art data view\web_test\static/img/QL532768D28.jpg
D:\workspace\Current project\Art data view\web_tes